<a href="https://colab.research.google.com/github/silvia-jesus/CAR_Labrea/blob/main/CAR_RL_Labrea_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC')

Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
from osgeo import gdal

In [ ]:
! pip install geopandas
import geopandas as gpd

/usr/local/lib/python3.10/dist-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-5-115f0ca902c9>:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

In [ ]:
!sudo apt install libspatialindex-dev
!pip install rtree
!pip install pygeos

import rtree
import pygeos
gpd.options.use_pygeos = True
USE_PYGEOS=1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libspatialindex-dev is already the newest version (1.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
!pip install rasterio
import rasterio as rio

In [ ]:
!pip install rasterstats
from rasterstats import zonal_stats

In [ ]:
from rasterio import mask

In [ ]:
!pip install fiona
import fiona

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
import glob
import shutil
import os
import time

In [ ]:
mapbiomas=glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/*tif')
mapbiomas

['/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/mapbiomas_2020_102033.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/mapbiomas_2008_102033.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/mapbiomas_2012_102033.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/mapbiomas_2016_102033.tif']

In [ ]:
recode6=glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/*')
recode6

['/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2020_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2008_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2012_recode.tif',
 '/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS_RECODE/2016_recode.tif']

In [ ]:
files_rl = glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_SPLIT/*.shp')
len(files_rl)

2860

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC/Arrays_LULC'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC/Arrays_LULC')


Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
for rl in files_rl:
    for lulc in mapbiomas:
        with fiona.open(rl, 'r') as shapefile:
            shapes = [feature['geometry'] for feature in shapefile]
        with rio.open(lulc) as src:
            out_image, transformed = rio.mask.mask(src, shapes, crop=True, filled=True)

            gdf = gpd.read_file(rl)

            id_rl=gdf.iloc[0]['IDF']
            year = lulc.replace('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/MAPBIOMAS/mapbiomas_','').replace('_102033.tif','')

            data=out_image.flatten()
            data=data[data >= 0] #to remove nan
            unique, counts = np.unique(data, return_counts=True)
            arr = np.array([unique, counts])
            df = pd.DataFrame(data=arr)
            new_header = df.iloc[0] #grab the first row for the header
            df = df[1:] #take the data less the header row
            df.columns = new_header #set the header row as the df header

            df['IDF']=id_rl
            df['ano']=year
            df.to_csv(f'/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC/Arrays_LULC/lulc_{id_rl}_{year}.csv')

o processamento anterior demorou 2h15m16s

In [ ]:
lulc_csv = glob.glob('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/RL_LULC/Arrays_LULC/lulc_*.csv')

In [ ]:
lulc_df = pd.concat(map(pd.read_csv, lulc_csv), ignore_index=True)
lulc_df = lulc_df.iloc[:, 2:] #remove old index and 0 values

cols = lulc_df.columns
cols

In [ ]:
lulc_df = lulc_df[['IDF', 'ano', '3', '15', '11', '12', '33', '4', '41']]
lulc_df = lulc_df.fillna(0)
lulc_df.head(6)

In [ ]:
value = pd.read_csv('/content/drive/MyDrive/CAR_RL_Labrea/DADOS/code_mapbiomas_legend.csv',sep=';',encoding='latin1')
value['value'] = value['value'].astype(str)
value.head()

In [ ]:
val_dict = dict(zip(value.value, value.name))
val_dict

In [ ]:
lulc_df = pd.concat(map(pd.read_csv, lulc_csv), ignore_index=True)
lulc_df = lulc_df.iloc[:, 2:] #remove old index and 0 values
cols = lulc_df.columns
cols

In [ ]:
lulc_df.rename(columns = val_dict, inplace = True)
lulc_df.head()

In [ ]:
if not os.path.exists('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC'):
  os.makedirs('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC')


Path('./').mkdir(parents=True, exist_ok=True)

In [ ]:
lulc_df.to_csv('/content/drive/MyDrive/CAR_RL_Labrea/OUTPUT/Tabelas_LULC/RL_LULC_Labrea.csv')